# Welcome

Oh hey.



# Markov Chains

A Markov chain is a special case of a Markov decision process with no actions. Specifically, there is a transition distribution $p(S_{t+1}|S_t)$. Given an initial distribution, $P(S_0)$, we get a distribution over trajectories by:
$$
p(S_0,S_1,\ldots,S_T) = p(S_0)p(S_1|S_0)\cdots p(S_T|S_{T-1})
$$

Note that for any MDP and any policy, $\pi$, there is an associated Markov chain defined by 
$$p_{\pi}(s'|s) = \sum_{a\in\mathcal{A}} p(s'|s,a)\pi(a,|s)$$

The transition probability of a Markov chain over a finite state space can be represented by a matrix:
$$
P(i,j) = p(s'=j|s=i).
$$

# Theory Question

Let $S_0,S_1,S_2,\ldots$ be a sequence of states generated by the Markov chain associated with $P\in \mathbb{R}^{n\times n}$.

Assume that the Markov chain always reaches a terminal state. Specifically, assume that there is a set $\mathcal{S}_{\mathrm{term}} \subset \{1,\ldots,n\}$ such that $p(\hat s | \hat s) = 1$ for all $\hat s\in \mathcal{S}_{\mathrm{term}}$ and there is an integer $H$ such that $p(S_t \in \mathcal{S}_{\mathrm{term}}| S_0=i) > 0 $ for all $t\ge H$ and all $i=1,\ldots, n$.



Without loss of generality, assume that $\mathcal{S}_{\mathrm{term}} = \{m+1,\ldots,n\}$ for some $m<n$. In this case $P$ can be expressed as
$$
P = \begin{bmatrix}
Q & B \\
0 & I
\end{bmatrix}.
$$

Show that for $i,j \in \{1,\ldots,m\}$ we have that $p(S_t = j| S_0=i)$ is the $(i,j)$ coordinate of $Q^t$.

Write your solution here or attached a hand-written note.

# Theory Question

Show that there is a number $\alpha \in (0,1]$ such that $p(S_{kH}\notin \mathcal{S}_{\mathrm{term}} | S_0=i)\le (1-\alpha)^k$ for all $i\in \{1,\ldots,m\}$.

Hints:
* First examine the $k=1$ case and use induction
* In general we have $p(S_{kH}\notin \mathcal{S}_{\mathrm{term}} | S_0=i)=\sum_{j=1}^m p(S_{kH}=j | S_0=i)$

Write your solution here or attached a hand-written note.

# Theory Question

Show that if $\lambda$ is an eigenvalue of $Q$ then $|\lambda| < 1$.

Write your solution here or attached a hand-written note.

# Theory Question

Let $r$ be a vector in $\mathbb{R}^n$.
Show that if $r(i)=0$ and $v(i)=0$ for all $i\in \mathcal{S}_{\mathrm{term}}$, the there are unique values $v(1),\ldots,v(m)$ such that
$$
v = r + Pv
$$

Write your solution here or attached a hand-written note.

# OpenAI Gym

Today we will be working with a simple problem called "Frozen Lake" from OpenAI Gym. The discussion from class of MDPs with terminal states was based on this example. It is a text-based maze environment that your controller will learn to navigate. It is slippery, however, so sometimes you don't always move where you try to go.

You can install OpenAI Gym via

```
pip install gym
```


In [ ]:
import gym
import numpy as np
import scipy.linalg as la

# This loads the environment
env = gym.make('FrozenLake-v0')
# This shows the basic layout
env.render()

The board can be interpreted as follows
* S - Start State
* F - Frozen parts
* H - Holes
* G - Goal

The colored square corresponds to the current state.

The episode ends if you hit a hole or the goal state. You recieve a reward of $1$ if you reach the goal and a reward of $0$ otherwise. 

Below is a  simulation of the system with randomly generated actions. It runs until the environment says it is `done`.

In [ ]:
s = env.reset()

done = False
R = []
while not done:
    a = env.action_space.sample()
    s, r, done, info = env.step(a)
    R.append(r)

TimeLimitReached = False
if 'TimeLimit.truncated' in info.keys():
    if info['TimeLimit.truncated'] == True:
        TimeLimitReached = True
    
env.render()
print('Time Limit Reached?',TimeLimitReached)
print('Sequence of Rewards:')
print(R)

By default, the environment has a time limit of 100 steps. After this many steps, the environment will return a `done` value of `True`, even if the terminal state has not been reached. For example, if you try moving up the whole time, this will probably happen. This is important to keep in mind since:
* If we were trying to determine the terminal states just based on simulation data, we should not mark non-terminal states as terminal. 
* If we are estimating the value obtained of a particular policy, if we count these terminated trajectories, we will get an inaccurate value because the trajectory should have continued past the time limit.

In [ ]:
# Run this. It will probably reach the time limit.
s = env.reset()

done = False
R = []
while not done:
    # Always try to move up
    a = 3
    s, r, done, info = env.step(a)
    R.append(r)

TimeLimitReached = False
if 'TimeLimit.truncated' in info.keys():
    if info['TimeLimit.truncated'] == True:
        TimeLimitReached = True
    
env.render()
print('Time Limit Reached?',TimeLimitReached)
print('Sequence of Rewards:')
print(R)
print('The total steps is',len(R))

# Extracting the Exact MDP

For the simple text-based examples, we can extract MDP exactly. We can use this to test basic algorithms like policy iteration and value iteration. The code below computes `P_true`, `R_true`, `S_term`, and `S_non_term`.

* `P_true[i,a,j]` encodes $p(S'=j|S=i,A=a)$
* `R_true[i,a]` encodes $\mathbb{E}[R_1 | S_0=i,A_0=a]$
* `S_term` is a list of the terminal states
* `S_non_term` is a list of the states that are not terminal

In [ ]:
# Build it exactly
env = gym.make('FrozenLake-v0')
env.reset()

nS = env.observation_space.n
nA = env.action_space.n

P_true = np.zeros((nS,nA,nS))
R_true = np.zeros((nS,nA))
S_term = []

for s in range(nS):
    for a in range(nA):
        transitions = env.P[s][a]
        for p,s_next,r,done in transitions:
            P_true[s,a,s_next] += p
            R_true[s,a] += p*r
            
            if done:
                if s_next not in S_term:
                    S_term.append(s_next)
            
            
S_term = list(set(S_term))
S_non_term = list(set(range(nS)) - set(S_term))


# Coding Question

Code up the policy iteration algorithm for the undiscounted case with terminal states. Compute an optimal policy, $\pi^\star$ and the optimal value function $V^\star$. Display the values of $V^\star$ so that I can check it. 

In [ ]:
# Put your code here

# Coding Question

Run 1000 simulations using the optimal and display the average total return obtained by these simulations. It should be close to $V^*(0)$.

Note, only include returns corresponding to simulations that did not reach the time limit in the average. If you include those in which the time limit is reached, you will bias the average.

In [ ]:
# Put your code here